In [32]:
import json
import pandas as pd
from datasets import Dataset
import os

file_path = "SAT_raw_predictions_gemini_2.5_flash_0.csv"

df = pd.read_csv(file_path)

SAT_DATASET_FOLDER = "/ocean/projects/cis250208p/shared/datasets/SAT"
data = []

with open(os.path.join(SAT_DATASET_FOLDER, "SAT_labeled.jsonl"), "r") as f:
    for line in f:
        data.append(json.loads(line))

ds = Dataset.from_list(data)

ds = ds.select(range(100))

In [33]:
match_list = []

for i in range(len(df)):
    pred = df.iloc[i]["prediction"]
    if not isinstance(pred, str) or len(pred.strip()) == 0:
        match_list.append(False)
        continue

    pred_index = ord(pred) - ord('A')
    if pred_index < 0 or pred_index >= len(ds[i]["answer_choices"]):
        match_list.append(False)
        continue
    answer = ds[i]["answer_choices"][pred_index]
    if answer == ds[i]["correct_answer"]:
        match_list.append(True)
    else:
        match_list.append(False)

print(match_list)

[False, True, False, False, True, True, False, True, True, True, True, True, False, True, True, False, False, True, True, True, True, True, False, True, True, True, True, True, True, True, False, False, False, True, True, False, False, True, False, True, True, False, True, False, True, True, False, True, True, False, True, True, True, True, True, True, False, False, True, True, False, False, False, True, True, False, True, False, True, False, True, True, True, False, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, False, True, False, True, False, False, True, False, True, True, False]


In [34]:
# count the number of matches
matched = sum(match_list)
print(f"Matched {matched} out of {len(df)}")

# count the number of matches for each question
question_matches = {}


Matched 66 out of 100


In [3]:
print(df.head())

   index             category  \
0      0  DISTANCE_COMPARISON   
1      1    RELATIVE_LOCATION   
2      2    RELATIVE_LOCATION   
3      3    RELATIVE_LOCATION   
4      4    RELATIVE_LOCATION   

                                            question  \
0  Which point is closer to the camera taking thi...   
1  Considering the relative positions, is iron ch...   
2  Considering the relative positions, where is b...   
3  Considering the relative positions, is extenda...   
4  Considering the relative positions, where is s...   

                                           reasoning prediction answer  \
0  The image shows a perspective view where objec...          B      C   
1  The image shows a perspective view of a 3D sce...          A  below   
2  The image shows a single 3D object that appear...        NaN  right   
3  The image depicts a 3D scene with a vanishing ...          B  below   
4  The image shows a colorful landscape painting ...          A  right   

   is_correct      

In [ ]:
import json
import os

SAT_TRAIN_GENERATED_FOLDER = "/ocean/projects/cis250208p/shared/datasets/SAT/train_generated_1"

In [2]:
from PIL import Image
from io import BytesIO
import base64

def base64_to_image(base64_str: str) -> Image.Image:
    """Convert a base64 string back to a PIL Image."""
    img_data = base64.b64decode(base64_str)
    return Image.open(BytesIO(img_data))

def deserialize_messages(serialized):
    """Convert base64-encoded images in messages back to PIL Images."""
    deserialized = []
    for msg in serialized:
        new_msg = {"role": msg["role"], "content": []}
        for item in msg["content"]:
            if item["type"] == "image" and "data" in item:
                # Reconstruct PIL image from base64 data
                image = base64_to_image(item["data"])
                new_msg["content"].append({
                    "type": "image",
                    "image": image
                })
            else:
                new_msg["content"].append(item)
        deserialized.append(new_msg)
    return deserialized

In [3]:
data = []
with open(os.path.join(SAT_TRAIN_GENERATED_FOLDER, "conversation_1.jsonl"), "r") as f:
    for line in f:
        data.append(json.loads(line))

deserialized = deserialize_messages(data[0]["messages"])
print(deserialized)
# deserialized[0]["content"][0]["image"].save("test.png")

[{'role': 'user', 'content': [{'type': 'image', 'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=400x400 at 0x7F88C8511340>}, {'type': 'text', 'text': "\n### Situation Description\nGiven an image and a spatial reasoning question, we need to all entities that are included in the question.\n\n# Example 1\n[Question] You are standing at the airplane's position, facing where it is facing. Is the the person on your left or right?\n[Detect] [airplane, person]\n\n# Examples 2\n[Question] From the old man's perspective, is the person wearing a hat on the left of the green car?\n[Detect] [old man, person wearing a hat, green car]\n\n# Examples 3\n[Question] From the car's perspective, which is on the right side: the person or the tree?\n[Detect] [car, person, tree]\n\n### Your Task\nNow, given the question below, please identify the entities that are included in the question.\n\n[Question] From the camera's point of view, Considering the relative positions, is iron cheir above or b

In [37]:
from datasets import Dataset
import json
import os

SAT_DATASET_FOLDER = "/ocean/projects/cis250208p/shared/datasets/SAT"
data = []

with open(os.path.join(SAT_DATASET_FOLDER, "SAT_labeled.jsonl"), "r") as f:
    for line in f:
        data.append(json.loads(line))

ds = Dataset.from_list(data)

SAT_TRAIN_GENERATED_FOLDER = "/ocean/projects/cis250208p/shared/datasets/SAT/train_generated_1"

correct_idx = []
total_count = 0

for root, _, files in os.walk(SAT_TRAIN_GENERATED_FOLDER):
    total_count = len(files)
    for filename in files:
        idx = int(filename.split("_")[-1].split(".")[0])
        file_path = os.path.join(root, filename)
        with open(file_path, "r") as f:
            data = []
            for line in f:
                data.append(json.loads(line))

            try:
                answer_extracted = data[-1]["response"].split("[Answer]")[1].strip()
                answer_extracted = answer_extracted.split(".")[0].strip()
            except:
                continue

        
            try:
                pred_index = ord(answer_extracted) - ord('A')
            except:
                print(answer_extracted)
                continue

            if pred_index < 0 or pred_index >= len(ds[idx]["answer_choices"]):
                continue
            answer = ds[idx]["answer_choices"][pred_index]
            if answer == ds[idx]["correct_answer"]:
                correct_idx.append(idx)

print(len(correct_idx)/total_count)
print(len(correct_idx))
print(total_count)
            
                

The question cannot be answered as there are no separate gray rectangle sofa and green threecushion sofa in the image
Cannot answer
Cannot answer the question as there is no doorway in the image
The question cannot be answered as posed because the red and green sections are part of a single object and are positioned side-by-side on the same horizontal plane, not one above or below the other
No square cardboard box is visible in the image
No brown wooden chair (marked b) is visible in the image
Cannot answer
The question cannot be answered as posed because there is no point a in the image
There is no black chair with round backrest (marked a) in the image
Cannot answer
Neither A nor B can be definitively chosen as the objects are at the same depth
The question cannot be answered as there is no doorway in the image
0.642023346303502
495
771
